In [1]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [2]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import transforms
import os
from PIL import Image
import torch
import ssl
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import splitfolders
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

def is_folder_empty(folder_path):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"The folder '{folder_path}' does not exist.")
        return False

    # Check if the folder is empty
    for root, dirs, files in os.walk(folder_path):
        if not dirs and not files:
            print(f"The folder '{root}' is empty.")
            return True  # The folder is empty

    print(f"The folder '{folder_path}' is not empty.")
    return False

def get_subdirectories(folder_path):
    subdirectories = []
    for entry in os.listdir(folder_path):
        entry_path = os.path.join(folder_path, entry)
        if os.path.isdir(entry_path):
            subdirectories.append(entry_path)
    return subdirectories


def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

ssl._create_default_https_context = ssl._create_unverified_context

if __name__ == '__main__':
    # Define the device to be used for training
    device = torch.device("cuda")

    # Set up the transform to resize and normalize the images
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5]),
    ])

    # Update input folder and output folder paths
    input_folder = r"/kaggle/input/images-highpassfilter-small/IMAGES_HighPassFilter_Small"
    output_folder = r"/kaggle/working/Images"

    ### Uncomment only for first time. once data is splitted into train and validation, comment it out
    splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.2), group_prefix=None)

    # Check for empty folders in the training and validation data
    train_path = os.path.join(output_folder, 'train')
    val_path = os.path.join(output_folder, 'val')

    train_directories = get_subdirectories(train_path)
    val_directories = get_subdirectories(val_path)

    # Create datasets for the training and testing sets
    train_dataset = torchvision.datasets.ImageFolder(output_folder + '/train', transform=transform)
    val_dataset = torchvision.datasets.ImageFolder(output_folder + '/val', transform=transform)
    train_size = len(train_dataset)
    val_size = len(val_dataset)

    # Create the data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True,num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=25, shuffle=True,num_workers=4)
    list_of_classes = os.listdir(r"/kaggle/working/Images/train")
    print(list_of_classes)
    classes = list(train_dataset.class_to_idx.keys())
    classes.sort()

from sklearn.metrics import accuracy_score, precision_score, recall_score







# Define the ResNet model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(4096, len(classes))
)
model = model.to(device)  # move the model to the specified device

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
START_LR = 0.0001
optimizer = optim.Adam(model.parameters(), lr=START_LR)
criterion = criterion.to(device)

# Train and Validate the model
for epoch in range(10):
    torch.cuda.empty_cache()
    print('Epoch {}/{}'.format(epoch + 1, 10))
    print('-' * 10)

    # Training phase
    model.train()
    running_loss = 0
    running_corrects = 0
    predictions = []
    true_labels = []
    all_labels = []
    all_predictions = []

    for inputs, labels in tqdm(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        true_labels.extend(labels.cpu().numpy())

        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(preds.cpu().numpy())
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)

     # Calculate accuracy, precision, and recall for all classes
    acc = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average=None)
    recall = recall_score(all_labels, all_predictions, average=None)

    # Calculate accuracy for each class
    acc_per_class = {}
    for i, class_name in enumerate(classes):
        class_indices = [idx for idx, lbl in enumerate(true_labels) if lbl == i]
        class_true_labels = [true_labels[idx] for idx in class_indices]
        class_preds = [predictions[idx] for idx in class_indices]

        class_acc = accuracy_score(class_true_labels, class_preds)
        acc_per_class[class_name] = class_acc

    print('Accuracy for all classes: ', acc_per_class)
    print("")
        
    print('Precision for all classes: ', precision)
    print("")
    print('Recall for all classes: ', recall)
    print("")
    
    report_dict = classification_report(true_labels, predictions, target_names=classes, output_dict=True)
    report_pd = pd.DataFrame(report_dict)
    report_pd.to_csv('training-classification-epoch' + str(epoch + 1) + '.csv')
    
    cnf_matrix = confusion_matrix(true_labels, predictions)
    df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index=classes, columns=classes)
    df_cm.to_csv('confusion-matrix-train-epoch' + str(epoch + 1) + '.csv')
    
    print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    # Validation phase
    running_loss = 0
    running_corrects = 0
    predictions = []
    true_labels = []

    model.eval()
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.extend(labels.cpu().numpy())

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())

            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(val_dataset)
    epoch_acc = running_corrects.double() / len(val_dataset)

    report_dict = classification_report(true_labels, predictions, target_names=classes, output_dict=True)
    report_pd = pd.DataFrame(report_dict)
    report_pd.to_csv('val-classification-epoch' + str(epoch + 1) + '.csv')

    cnf_matrix = confusion_matrix(true_labels, predictions)
    df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index=classes, columns=classes)
    df_cm.to_csv('confusion-matrix-val-epoch' + str(epoch + 1) + '.csv')

    print('Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    torch.cuda.empty_cache()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Copying files: 8689 files [02:34, 56.41 files/s] 


['afrthr1', 'pabspa1', 'blnwea1', 'mouwag1', 'bkctch1', 'scrcha1', 'gbesta1', 'blhgon1', 'varsun2', 'crheag1', 'chucis1', 'wtbeat1', 'gobbun1', 'afrgrp1', 'whbtit5', 'mabeat1', 'vimwea1', 'hunsun2', 'whbwea1', 'stusta1', 'grwpyt1', 'kvbsun1', 'whbwhe3', 'squher1', 'hadibi1', 'fotdro5', 'yebere1', 'grbcam1', 'categr', 'fislov1']


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 282MB/s]


Epoch 1/10
----------


100%|██████████| 278/278 [01:00<00:00,  4.57it/s]


...

Val Loss: 0.6582 Acc: 0.8434
Epoch 10/10
----------


100%|██████████| 278/278 [00:55<00:00,  5.02it/s]


Accuracy for all classes:  {'afrgrp1': 0.9253731343283582, 'afrthr1': 0.9632034632034632, 'bkctch1': 0.9359504132231405, 'blhgon1': 0.9339622641509434, 'blnwea1': 0.95, 'categr': 0.9785714285714285, 'chucis1': 0.9090909090909091, 'crheag1': 0.9785714285714285, 'fislov1': 0.7560975609756098, 'fotdro5': 0.9728682170542635, 'gbesta1': 0.9365079365079365, 'gobbun1': 0.9528301886792453, 'grbcam1': 0.97265625, 'grwpyt1': 0.927710843373494, 'hadibi1': 0.9757575757575757, 'hunsun2': 0.6538461538461539, 'kvbsun1': 0.6666666666666666, 'mabeat1': 0.9305555555555556, 'mouwag1': 0.9518072289156626, 'pabspa1': 0.8, 'scrcha1': 0.9696969696969697, 'squher1': 0.8285714285714286, 'stusta1': 0.8, 'varsun2': 0.9619047619047619, 'vimwea1': 0.8409090909090909, 'whbtit5': 0.8333333333333334, 'whbwea1': 0.8846153846153846, 'whbwhe3': 0.9863013698630136, 'wtbeat1': 0.9550561797752809, 'yebere1': 0.9101796407185628}

Precision for all classes:  [0.88571429 0.97587719 0.93402062 0.94285714 0.91935484 0.97558494


100%|██████████| 70/70 [00:09<00:00,  7.27it/s]

Val Loss: 0.6305 Acc: 0.8594
